# References
- https://www.w3schools.com/sql/sql_intro.asp
- O'REILLY Learning SQL Generate, Manipulate, and Retrieve Data Alan Beaulieu
- O'REILLY Using SQLITE Jay A. Kreibich

Programmes:
- https://sqlitebrowser.org/
- https://www.youtube.com/watch?v=wGqVjwNpBxY  SQLITE en ligne de commandes

In [ ]:
# Standard library import
from pathlib import Path

root = Path.home()

# Globals affectation
DATABASE_NAME = 'essai.db'
ATELIERS = ['MEP','Wet','PECVD','PVD','Serig','Sinton']
FOLDER =  root / Path('PV_DB_management_files')

In [ ]:
def df2sqlite(dataframe, path_db=None, tbl_name="import"):

    '''The function df2sqlite converts a dataframe into a squlite database.
    
    Args:
       dataframe (panda.DataFrame): the dataframe to convert in a data base
       path_db (Path): full pathname of the database
       tbl_name (str): name of the table
    '''
    
    # Standard library imports
    import sqlite3
    
    # 3rd party imports
    import pandas as pd
    

    if path_db is None:  # Connection to the database
        conn = sqlite3.connect(":memory:")
    else:
        conn = sqlite3.connect(path_db)
        
    # Creates a database and a table
    cur = conn.cursor()
    col_str = '"' + '","'.join(dataframe.columns) + '"'
    cur.execute(f"CREATE TABLE IF NOT EXISTS {tbl_name} ({col_str})")
    dataframe.to_sql(tbl_name, conn, if_exists='replace', index = False)

    cur.close()
    conn.close()
    
def create_parsing_db():
    
    '''Convert the list of .xlsx file of the parsing FOLDER into a databae in which
    each .xlsx file corresponds to a table.
    
    Args:
    
    Note: the function uses the global
       FOLDER (str): full path of de FOLDER parsing or filt_<i>
    '''
    
    # Standard library imports
    import os
    
    # 3rd party imports
    import pandas as pd

    for file in [file for file in os.listdir(FOLDER) if file.endswith('.xlsx')]:
        df = pd.read_excel(os.path.join(FOLDER,file))
        if len(df) > 0:
            df2sqlite(df, path_db=os.path.join(FOLDER,DATABASE_NAME), tbl_name=file.split('.')[0])
        else: 
            print(f'Warning: the file {file} is empty and will not be added to the database')

def query_db(query):
    
    '''Sends a query to the database.
    
    Args:
       query (str): the sql query
       
    Retuns:
       The result of the query as a dataframe.
    '''
    
    # Standard library import
    import os
    import sqlite3
    
    # 3rd party import
    import pandas as pd
    
    conn = sqlite3.connect(os.path.join(FOLDER,DATABASE_NAME))
    cur = conn.cursor()
    
    try:
        cur.execute(query)
        
    except sqlite3.OperationalError as e:
        raise Exception(f'An errors has occured in the query: {e}')
    else:
        result_querry = {i:list(x) for i,x in enumerate(cur.fetchall())}
        df = pd.DataFrame.from_dict(result_querry).T    
    finally:  
        cur.close()
        conn.close()
    
    return df

def check_id_existance(table,PK):
    
    '''Returns True if the key PK allready exits in the table Returns False ortherwise.
    '''
    
    
    # Standard library import
    import os
    import sqlite3
    from string import Template
    
    
    conn = sqlite3.connect(os.path.join(FOLDER,DATABASE_NAME))
    cur = conn.cursor()
    
    template = Template('''SELECT * FROM $table1
                           WHERE  PK='$PK'
                        ''')
    id_exists = False
    try:
        cur.execute(template.substitute({'table1': table,
                                         'PK': PK,}))
    except sqlite3.OperationalError as e:
        raise Exception(f'An errors has occured in the query: {e}')
    else:
        row = cur.fetchone() 
    finally:  
        cur.close()
        conn.close()
    
    if row: 
        id_exists = True 
    else:
        id_exists = False
    return id_exists


def insert_value(table,value):
    
    # Standard library import
    import os
    import sqlite3
    from string import Template
    
    
    conn = sqlite3.connect(os.path.join(FOLDER,DATABASE_NAME))
    cur = conn.cursor()
    
    template = Template('''INSERT INTO $table
                           VALUES $value
                        ''')
    
    cur.execute(template.substitute({'table': table,
                                     'value': value,}))
    
    conn.commit()
    
    cur.close()
    conn.close()
    


def delete_multiple_records(table,idList):
    
    # Standard library imports
    import os
    import sqlite3
    from string import Template
    
    template = Template('''DELETE from $table
                           WHERE PK = ?
                        ''')
    
    try:
        conn = sqlite3.connect(os.path.join(FOLDER,DATABASE_NAME))
        cur = conn.cursor()
        cur.executemany(template.substitute({'table': table,}),idList)
        conn.commit()
        print("Total", cur.rowcount, "Records deleted successfully")
        cur.close()

    except sqlite3.Error as error:
        print("Failed to delete multiple records from sqlite table", error)
    finally:
        if conn:
            conn.close()

def look_db_schema():
    '''
    https://stackoverflow.com/questions/604939/
    how-can-i-get-the-list-of-a-columns-in-a-table-for-a-sqlite-database#:~:text=To%20get%20column%20information%20you%20can%20use%20the,information%20on%20the%20pragma%20statements%2C%20see%20the%20documentation.
    '''
    
    query = '''SELECT m.name as tableName, 
                      p.name as columnName
                FROM sqlite_master m
                left outer join pragma_table_info((m.name)) p
                on m.name <> p.name
                order by tableName, columnName;
             '''
    df = query_db(query) 
    schema = {atelier:df.loc[col,1].tolist() for atelier, col in df.groupby(0).groups.items()}
    
    return schema

def select_all_table(table_name):
    from string import Template
    
    template = Template('''SELECT * FROM $table
                        ''')
    query = template.substitute({'table': table_name,})
    
    df = query_db(query) 
    
    return df
    
    
    

In [ ]:
# Builds 6 fake excel files and a databse with six corresponding tables with N°Demande <= n_demandes

# Standard library imports
import datetime
import itertools
import random
import string
from pathlib import Path
from datetime import date

# 3rd party imports
import numpy as np
import pandas as pd



random_string_generator = lambda n:[''.join(random.choice(string.ascii_letters) for x in range(str_size)) for _ in range(n)]
 
str_size = 12
 
n_demandes = 50 # number of different demands
for atelier in ATELIERS:
    dic = {'N°demande':(n_demande := list(itertools.chain(*[[i]*random.randint(1,5) for i in range(n_demandes)]))),
       'Version':[random.randint(0, 2) for _ in range(len(n_demande))],
       'Split':[random.randint(0, 1) for _ in range(len(n_demande))],
       'Date' : [(date.today() + datetime.timedelta(days=random.randint(0, 5))).isoformat() for _ in range(len(n_demande))],
       'P1': np.random.normal(loc = 0.0, scale = 1.0, size = len(n_demande)),
       'P2': np.random.normal(loc = 0.0, scale = 1.0, size = len(n_demande)),  
       'P3': np.random.normal(loc = 0.0, scale = 1.0, size = len(n_demande)),
       'P4': np.random.normal(loc = 0.0, scale = 1.0, size = len(n_demande)),
       'PX': np.random.normal(loc = 0.0, scale = 1.0, size = len(n_demande)),
       'Commentaire': random_string_generator(len(n_demande))}
    
    dg = pd.DataFrame.from_dict(dic)
    dg = dg.drop_duplicates(['N°demande','Version','Split'])
    dg['PK'] = dg.apply(lambda row:f'{row[0]}_{row[1]}_{row[2]}',axis=1)
    dg.to_excel(Path(FOLDER) / Path(atelier+'.xlsx'),index=False)
    
create_parsing_db()

In [ ]:
# Check if if the primary key already exists

PK = '2_1_0'
for atelier in ATELIERS:
    print(atelier,check_id_existance(atelier,PK))

In [ ]:
# Insert a new value in a table in and only if the primary key doesn't exist

atelier = 'MEP'

dic = {'N°demande':101,
       'Version':1,
       'Split':1,
       'Date' : ( date.today() + datetime.timedelta(days=random.randint(0, 5))).isoformat(),
       'P1': 0.1,
       'P2': 0.1,  
       'P3': 0.5,
       'P4': 0.6,
       'PX': 0.7,
       }
dic['PK'] = f"{dic['N°demande']}_{dic['Version']}_{dic['Split']}"

if not check_id_existance(atelier,dic['PK']):

    insert_value(atelier,tuple(dic.values()))
    

In [ ]:
schema_dict = look_db_schema()
schema_dict

In [ ]:
# Delete records from a table

atelier = 'MEP'
ids_to_delete = [('101_1_1',), ('0_1_1',), ('0_2_1',)]
delete_multiple_records(atelier,ids_to_delete)

In [ ]:
# Select values
# SELECT PECVD.PK, Sinton.PK, Sinton.P1 FROM PECVD  LEFT JOIN Sinton ON PECVD.PK=Sinton.PK
#
#CREATE VIEW SYNTHESIS AS 
#SELECT PK,P1,P2 FROM  PECVD;

from pathlib import Path

query1 = '''SELECT 
              Sinton.P1 AS SINTON_P1,
              MEP.P1 AS MEP_P1,
              PECVD.P1 AS PECVD_P1,
              Wet.P2 AS Wet_P2,
              MEP.PK AS MEP_PEK,
              Sinton.PK AS Sinton_PK,
              PECVD.PK AS PECVD_PK,
              Wet.PK AS Wet_PK
           FROM 
              Sinton 
           JOIN MEP 
              ON Sinton.PK=MEP.PK 
           JOIN PECVD 
              ON Sinton.PK=PECVD.PK 
           JOIN Wet 
              ON Sinton.PK=Wet.PK
           WHERE
              Sinton.P1>0.5 AND Wet.P2>0.5
        '''

script = 'demo_tri.sql'
sql_file = Path(FOLDER) / Path(script)
with open(sql_file,'r') as file:
    query = ''.join(file.readlines())

df = query_db(query)
df.columns = ['SINTON_P1',' MEP_P1','PECVD_P1','Wet_P2','MEP_PEK','Sinton_PK','PECVD_PK','Wet_PK']
df

In [ ]:
print(df.to_markdown(tablefmt="grid",index=True))

In [ ]:
def format_excel_tab(df,tab_name,colour="green"):
    """

    :param df: name of dataframe
    :param tab_name: tab in df to be formatted
    :param colour: colour for tab header + column header
    :return: selected tabs is formatted, inplace
    """
    worksheet = writer.sheets[tab_name]
    worksheet.set_tab_color(colour)
    auto_width(df,tab_name)
    header_format(df,tab_name,colour)

def auto_width(df,tab):
    """


    :param df: name of dataframe
    :param tab: tab in df with improper width
    :return: selected tabs' columns' width is auto set, inplace
    """
    for column in df:
        column_width = max(df[column].astype(str).map(len).max(), len(column))
        col_idx = df.columns.get_loc(column)
        writer.sheets[f'{tab}'].set_column(col_idx, col_idx, column_width)

def header_format(df,tab,colour="green"):
    """

    :param df: name of dataframe
    :param tab: tab in df with improper width
    :return:selected tabs' columns' headers are coloured, inplace
    """
    header_format = workbook.add_format({
        'bold': True,
        'text_wrap': True,
        'valign': 'top',
        'fg_color': colour,
        'border': 1,
        'align':'center'})

    for col_num, value in enumerate(df.columns.values):
        writer.sheets[f'{tab}'].write(0, col_num, value, header_format)

In [ ]:
saved_filename="Movies_Per_Category.xlsx"
with pd.ExcelWriter(saved_filename, engine='xlsxwriter') as writer:

    workbook  = writer.book

    g_movies.to_excel(writer, sheet_name='G',index=False)
    format_excel_tab(g_movies,"G","orange")

    r_movies.to_excel(writer, sheet_name='R',index=False)
    format_excel_tab(pg_movies,"R","red")

    pg_movies.to_excel(writer, sheet_name='PG',index=False)
    format_excel_tab(pg_movies,"PG") #default green

    pg13_movies.to_excel(writer, sheet_name='PG-13',index=False)
    format_excel_tab(pg13_movies,"PG-13")

In [ ]:
import tkinter as tk
from tkinter import ttk
global count, entries_value

def add_record():
    global count, entries_value
    
    my_tree.tag_configure('oddrow',background='white')
    my_tree.tag_configure('evenrow',background='lightblue')
    tag = 'oddrow'
    if count%2:
        tag = 'evenrow'
    
    my_tree.insert(parent='',
                   index='end',
                   iid=count,
                   text='Parent',
                   values= [entry.get() for entry in entries_value],
                   tags=(tag,))
    count += 1

root = tk.Tk()
root.title('INES ATELIERS')
root.geometry("1400x500")

style = ttk.Style()

style.theme_use('clam')
style.configure('Treview',
                background='#D3D3D3',
                foreground="black",
                rowheight=25,
                fieldbackground='#D3D3D3')
style.map('Treeview',
          background=[('selected', 'green')])
my_tree = ttk.Treeview(root)
schema = look_db_schema()
my_tree['column'] = schema['PECVD']
my_tree.column('#0', width=0)
for name in schema['PECVD']:
       my_tree.column(name, anchor=tk.W, width=120)
        
my_tree.heading('#0', text='',anchor=tk.W)
for name in schema['PECVD']:
       my_tree.heading(name, text=name,anchor=tk.W)
df = select_all_table('PECVD')

my_tree.tag_configure('oddrow',background='white')
my_tree.tag_configure('evenrow',background='lightblue')

count = len(df)
for idx in df.index:
    tag = 'oddrow'
    if idx%2:
        tag = 'evenrow'
    my_tree.insert(parent='',
                   index='end',
                   iid=idx,
                   text='Parent',
                   values=df.iloc[idx].tolist(),
                   tags=(tag,))

my_tree.pack(pady=20)

add_frame = tk.Frame(root)
add_frame.pack(pady=20) 

entries_value = []
for idx,name in enumerate(schema['PECVD']):
    tk.Label(add_frame,text=name).grid(row=0,column=idx)
    entry_value = tk.Entry(add_frame)
    entries_value.append(entry_value)
    entry_value.grid(row=1,column=idx)
       
tk.Button(root,text='Add Record',command=add_record).pack(pady=20)

root.mainloop()